# Demo of delta lake change data feed

## create a silver table for PRODUCT Dimension

In [0]:
product = [
    ("usa", "productA", 1000, 200),
    ("usa", "productB", 1500, 300),
    ("canada", "productA", 800, 150),
    ("canada", "productC", 1200, 250),
    ("mexico", "productB", 900, 180),
    ("mexico", "productC", 1100, 220)
]

Columns = ['Country','Product', 'Stock','Sales' ]

spark.createDataFrame(data = product, schema = Columns)\
    .write.format('delta').mode('overwrite')\
        .saveAsTable('silver_table')

In [0]:
%sql
select * from silver_table;

## Gnerate Gold table shpwing sales rate by country

In [0]:
import pyspark.sql.functions as F


In [0]:
spark.read.format('delta').table('silver_table')\
    .withColumn('SalesRate', F.round(F.col('Sales') / F.col('Stock'), 2))\
        .drop('Stock').drop('Sales')\
            .write.format('delta').mode('overwrite')\
                .saveAsTable('gold_Table')

In [0]:
%sql
SELECT * FROM silver_table;

In [0]:
%sql
SELECT * FROM gold_table;

## Enable chnage data feed on silver table

In [0]:
%sql
ALTER TABLE silver_table SET TBLPROPERTIES (delta.enableChangeDataFeed = true)

In [0]:
%sql
DESCRIBE HISTORY silver_table;

##  Update Silver table daily

In [0]:
# insert new record
product = [
    ("India", "productD", 3000, 600),
]

spark.createDataFrame(data = product, schema = Columns)\
    .write.format('delta').mode('append')\
        .saveAsTable('silver_table'  )

In [0]:
%sql
-- update a record

UPDATE silver_table SET Stock = '1000' WHERE Country  = 'canada' AND Product = 'productA';

In [0]:
%sql
DELETE FROM silver_table WHERE Country = 'mexico' AND Product ='productB';

In [0]:
%sql
SELECT * FROM silver_table;

EXPORE THE change data in SQL AND PYSPARK

In [0]:
%sql
DESCRIBE HISTORY silver_table;

In [0]:
%sql
SELECT * FROM table_changes('silver_table', 1) ORDER BY _commit_timestamp

In [0]:
change_df = spark.read.format('delta')\
    .option('readChangeData', True)\
        .option('startingVersion', 2)\
            .table('silver_table')

display(change_df)

## propogate changes from silver to gold table

In [0]:
%sql
 CREATE OR REPLACE TEMPORARY VIEW silver_table_latest_version AS
 SELECT * 
 FROM(
  SELECT * , RANK() OVER(PARTITION BY Country, Product ORDER BY _commit_version DESC) AS rnk 
  FROM table_changes('silver_table', 2, 5)
  WHERE _change_type != 'update_preimage'
 )
 WHERE rnk = 1; 

In [0]:
%sql
select * from silver_table_latest_version;

In [0]:
%sql
-- merge changes to gold layer
MERGE INTO gold_table AS t
  USING silver_table_latest_version AS s
  on t.Country = s.Country AND t.Product = s.Product
WHEN MATCHED AND s._change_type = 'update_postimage' 
  THEN UPDATE SET SalesRate = s.Sales / s.Stock
WHEN MATCHED AND s._change_type = 'delete'
  THEN DELETE
WHEN NOT MATCHED
  THEN INSERT ( Country, Product, SalesRate) 
    VALUES (s.Country, s.Product, s.Sales / s.Stock)


In [0]:
%sql
SELECT * FROM gold_table;